In [1]:
from keras.models import Sequential
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np

C:\Users\OW93790\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Read in DataFrame of stored images and types
images_df=pd.read_csv("Data/HAM10000_metadata_shuffle_reduced.csv")
images_df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0004421,ISIC_0029481.jpg,nv,follow_up,35.0,male,lower extremity
1,HAM_0006099,ISIC_0026311.jpg,bkl,consensus,55.0,female,face
2,HAM_0002810,ISIC_0031876.jpg,bkl,histo,70.0,female,chest
3,HAM_0002258,ISIC_0029220.jpg,bcc,histo,80.0,male,back
4,HAM_0004654,ISIC_0027130.jpg,mel,histo,70.0,male,chest


In [3]:
# Generate the trainer from the images and types using Keras
# Validation split chosen so that the datagens are not prime numbers and may be run in reasonable batches
datagen=ImageDataGenerator(rescale=1./255,validation_split=0.20)

# Reduce image size to reduce model storage size for gitlab
train_generator=datagen.flow_from_dataframe(dataframe=images_df, directory="Images\HAM10000_images", x_col="image_id", y_col="dx", subset="training", class_mode="categorical", target_size=(200,200), seed=42, batch_size=32, shuffle=True)

valid_generator=datagen.flow_from_dataframe(dataframe=images_df, directory="Images\HAM10000_images", x_col="image_id", y_col="dx", subset="validation", class_mode="categorical", target_size=(200,200), seed = 42, batch_size=32, shuffle=True)

Found 3648 images belonging to 7 classes.
Found 912 images belonging to 7 classes.


In [4]:
# Confirm classes of training and validation
print(f"Training classes: {list(set(train_generator.classes))}")
print(f"Validation classes: {list(set(valid_generator.classes))}")
# All classes are in both sets

Training classes: [0, 1, 2, 3, 4, 5, 6]
Validation classes: [0, 1, 2, 3, 4, 5, 6]


In [5]:
# Model setting code from https://medium.com/@vijayabhaskar96/tutorial-on-keras-flow-from-dataframe-1fd4493d237c
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(200,200,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
# added hidden layer for deep network
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))
model.compile(optimizers.rmsprop(lr=0.0001, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])

In [6]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
# STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=2
)

Epoch 1/2
114/114 [==============================] - 1173s 10s/step - loss: 1.8552 - acc: 0.2582 - val_loss: 1.7944 - val_acc: 0.3973
Epoch 2/2
114/114 [==============================] - 1131s 10s/step - loss: 1.7185 - acc: 0.3188 - val_loss: 1.6696 - val_acc: 0.3489


In [7]:
 # Save the model
model.save("Models/deep01.h5")

In [8]:
# STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=2
)

Epoch 1/2
114/114 [==============================] - 1157s 10s/step - loss: 1.6416 - acc: 0.3416 - val_loss: 1.6356 - val_acc: 0.4205
Epoch 2/2
114/114 [==============================] - 1061s 9s/step - loss: 1.6087 - acc: 0.3418 - val_loss: 1.5279 - val_acc: 0.3909


In [9]:
 # Save the model
model.save("Models/deep02.h5")

In [10]:
# STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=2
)

Epoch 1/2
114/114 [==============================] - 1068s 9s/step - loss: 1.5880 - acc: 0.3577 - val_loss: 1.5289 - val_acc: 0.4659
Epoch 2/2
114/114 [==============================] - 1062s 9s/step - loss: 1.5308 - acc: 0.3846 - val_loss: 1.4598 - val_acc: 0.4261


In [11]:
 # Save the model
model.save("Models/deep03.h5")

In [12]:
# STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=2
)

Epoch 1/2
114/114 [==============================] - 1068s 9s/step - loss: 1.5090 - acc: 0.3890 - val_loss: 1.3978 - val_acc: 0.4750
Epoch 2/2
114/114 [==============================] - 1084s 10s/step - loss: 1.4775 - acc: 0.3975 - val_loss: 1.4044 - val_acc: 0.4830


In [13]:
 # Save the model
model.save("Models/deep04.h5")

In [14]:
# STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=2
)

Epoch 1/2
114/114 [==============================] - 1088s 10s/step - loss: 1.4641 - acc: 0.4062 - val_loss: 1.3684 - val_acc: 0.5102
Epoch 2/2
114/114 [==============================] - 1093s 10s/step - loss: 1.4075 - acc: 0.4397 - val_loss: 1.3031 - val_acc: 0.5102


In [15]:
 # Save the model
model.save("Models/deep05.h5")

In [16]:
# STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=2
)

Epoch 1/2
114/114 [==============================] - 1092s 10s/step - loss: 1.3680 - acc: 0.4468 - val_loss: 1.3096 - val_acc: 0.4932
Epoch 2/2
114/114 [==============================] - 1094s 10s/step - loss: 1.3807 - acc: 0.4435 - val_loss: 1.2920 - val_acc: 0.5443


In [17]:
 # Save the model
model.save("Models/deep06.h5")

In [18]:
# STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=2
)

Epoch 1/2
114/114 [==============================] - 1102s 10s/step - loss: 1.3471 - acc: 0.4583 - val_loss: 1.2879 - val_acc: 0.5318
Epoch 2/2
114/114 [==============================] - 1087s 10s/step - loss: 1.3309 - acc: 0.4679 - val_loss: 1.2562 - val_acc: 0.5295


In [19]:
 # Save the model
model.save("Models/deep07.h5")

In [20]:
# STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=2
)

Epoch 1/2
114/114 [==============================] - 1112s 10s/step - loss: 1.2995 - acc: 0.4852 - val_loss: 1.2460 - val_acc: 0.5466
Epoch 2/2
114/114 [==============================] - 1091s 10s/step - loss: 1.3235 - acc: 0.4726 - val_loss: 1.2865 - val_acc: 0.5580


In [21]:
 # Save the model
model.save("Models/deep08.h5")

In [22]:
# STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=2
)

Epoch 1/2
114/114 [==============================] - 1101s 10s/step - loss: 1.2726 - acc: 0.5052 - val_loss: 1.2235 - val_acc: 0.5625
Epoch 2/2
114/114 [==============================] - 1099s 10s/step - loss: 1.2833 - acc: 0.5041 - val_loss: 1.2015 - val_acc: 0.5580


In [23]:
 # Save the model
model.save("Models/deep09.h5")

In [24]:
# STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=2
)

Epoch 1/2
114/114 [==============================] - 1104s 10s/step - loss: 1.2664 - acc: 0.5079 - val_loss: 1.2050 - val_acc: 0.5727
Epoch 2/2
114/114 [==============================] - 1101s 10s/step - loss: 1.2420 - acc: 0.5203 - val_loss: 1.2006 - val_acc: 0.5750


In [25]:
 # Save the model
model.save("Models/deep10.h5")

In [26]:
# STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=2
)

In [27]:
 # Save the model
model.save("Models/deep11.h5")

In [28]:
# STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=2
)

Epoch 1/2
114/114 [==============================] - 1096s 10s/step - loss: 1.2244 - acc: 0.5217 - val_loss: 1.2266 - val_acc: 0.5648
Epoch 2/2
114/114 [==============================] - 1100s 10s/step - loss: 1.2073 - acc: 0.5334 - val_loss: 1.2040 - val_acc: 0.5705


In [29]:
 # Save the model
model.save("Models/deep12.h5")

In [30]:
# STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=2
)

Epoch 1/2
114/114 [==============================] - 1102s 10s/step - loss: 1.2065 - acc: 0.5244 - val_loss: 1.1601 - val_acc: 0.5659
Epoch 2/2
114/114 [==============================] - 1086s 10s/step - loss: 1.1799 - acc: 0.5441 - val_loss: 1.1588 - val_acc: 0.5886


In [31]:
 # Save the model
model.save("Models/deep13.h5")

In [32]:
# STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=2
)

Epoch 1/2
114/114 [==============================] - 1103s 10s/step - loss: 1.1603 - acc: 0.5540 - val_loss: 1.2021 - val_acc: 0.5648
Epoch 2/2
114/114 [==============================] - 1088s 10s/step - loss: 1.1463 - acc: 0.5504 - val_loss: 1.1368 - val_acc: 0.5682


In [33]:
 # Save the model
model.save("Models/deep14.h5")

In [34]:
# STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=2
)

Epoch 1/2
114/114 [==============================] - 1101s 10s/step - loss: 1.1621 - acc: 0.5507 - val_loss: 1.1285 - val_acc: 0.5852
Epoch 2/2
114/114 [==============================] - 1108s 10s/step - loss: 1.1304 - acc: 0.5639 - val_loss: 1.1320 - val_acc: 0.5792


In [35]:
 # Save the model
model.save("Models/deep15.h5")

In [36]:
# STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=2
)

Epoch 1/2
114/114 [==============================] - 1092s 10s/step - loss: 1.0977 - acc: 0.5891 - val_loss: 1.1043 - val_acc: 0.5659
Epoch 2/2
114/114 [==============================] - 1092s 10s/step - loss: 1.1031 - acc: 0.5861 - val_loss: 1.1791 - val_acc: 0.5955


In [37]:
 # Save the model
model.save("Models/deep16.h5")

In [38]:
# STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=2
)

Epoch 1/2
114/114 [==============================] - 1103s 10s/step - loss: 1.0910 - acc: 0.5792 - val_loss: 1.1822 - val_acc: 0.5773
Epoch 2/2
114/114 [==============================] - 1097s 10s/step - loss: 1.0804 - acc: 0.5894 - val_loss: 1.1194 - val_acc: 0.6045


In [39]:
 # Save the model
model.save("Models/deep17.h5")

In [40]:
# STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=2
)

Epoch 1/2
114/114 [==============================] - 1144s 10s/step - loss: 1.0953 - acc: 0.5800 - val_loss: 1.1461 - val_acc: 0.5716
Epoch 2/2
114/114 [==============================] - 1055s 9s/step - loss: 1.0437 - acc: 0.6058 - val_loss: 1.1566 - val_acc: 0.5580


In [41]:
 # Save the model
model.save("Models/deep18.h5")

In [42]:
# STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=2
)

Epoch 1/2
114/114 [==============================] - 1053s 9s/step - loss: 1.0304 - acc: 0.6077 - val_loss: 1.1139 - val_acc: 0.6023
Epoch 2/2
114/114 [==============================] - 1038s 9s/step - loss: 1.0211 - acc: 0.6069 - val_loss: 1.1924 - val_acc: 0.5602


In [43]:
 # Save the model
model.save("Models/deep19.h5")

In [44]:
# STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=2
)

Epoch 1/2
114/114 [==============================] - 1054s 9s/step - loss: 1.0130 - acc: 0.6184 - val_loss: 1.0750 - val_acc: 0.6034
Epoch 2/2
114/114 [==============================] - 1201s 11s/step - loss: 0.9972 - acc: 0.6236 - val_loss: 1.1507 - val_acc: 0.5830


In [45]:
 # Save the model
model.save("Models/deep20.h5")

In [46]:
# STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=2
)

Epoch 1/2
114/114 [==============================] - 1272s 11s/step - loss: 0.9645 - acc: 0.6335 - val_loss: 1.1431 - val_acc: 0.6023
Epoch 2/2
114/114 [==============================] - 1226s 11s/step - loss: 0.9825 - acc: 0.6242 - val_loss: 1.0918 - val_acc: 0.5864


In [47]:
 # Save the model
model.save("Models/deep21.h5")

In [48]:
# STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=2
)

Epoch 1/2
114/114 [==============================] - 1348s 12s/step - loss: 0.9620 - acc: 0.6379 - val_loss: 1.1160 - val_acc: 0.5875
Epoch 2/2
 87/114 [=====================>........] - ETA: 4:15 - loss: 0.9354 - acc: 0.6462

KeyboardInterrupt: 

In [ ]:
 # Save the model
model.save("Models/deep22.h5")

In [ ]:
# STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=2
)

In [ ]:
 # Save the model
model.save("Models/deep23.h5")

In [ ]:
# STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=2
)

In [ ]:
 # Save the model
model.save("Models/deep24.h5")

In [ ]:
# STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=2
)

In [ ]:
 # Save the model
model.save("Models/deep25.h5")

In [ ]:
# Best model at 30 epochs